In [0]:
%pip install onnx onnxruntime-gpu
dbutils.library.restartPython()

In [0]:
#https://huggingface.co/blog/convert-transformers-to-onnx
import torch
from transformers import AutoModelForPreTraining, AutoTokenizer
DIRECTORY='/Volumes/edav_prd_cdh/cdh_ml/metadata_data/onnx_models'
# load model and tokenizer
# https://huggingface.co/docs/transformers/main/en/model_doc/distilbert#transformers.DistilBertForTokenClassification
model_user="sarahmiller137"
model_id = "distilbert-base-uncased-ft-ncbi-disease"
"""
Some weights of DistilBertForMaskedLM were not initialized from the model checkpoint at sarahmiller137/distilbert-base-uncased-ft-ncbi-disease and are newly initialized: ['vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_transform.weight']
"""
model = AutoModelForPreTraining.from_pretrained(f"{model_user}/{model_id}")
tokenizer = AutoTokenizer.from_pretrained(f"{model_user}/{model_id}")
dummy_model_input = tokenizer( "HuggingFace is a company based in Paris and New York", add_special_tokens=False, return_tensors="pt")
print(dummy_model_input.values())
onnxfile=f"{model_user}-{model_id}.onnx"
# export
torch.onnx.export(
    model, 
    tuple(dummy_model_input.values()),
    f=onnxfile,  
    input_names=['input_ids', 'attention_mask'], 
    output_names=['logits'], 
    dynamic_axes={'input_ids': {0: 'batch_size', 1: 'sequence'}, 
                  'attention_mask': {0: 'batch_size', 1: 'sequence'}, 
                  'logits': {0: 'batch_size', 1: 'sequence'}}, 
    do_constant_folding=True, 
    opset_version=13, 
)


In [0]:
%sh
cp *.onnx /Volumes/edav_dev_cdh/cdh_ml/metadata_data/onnx_models/
ls -ll /Volumes/edav_dev_cdh/cdh_ml/metadata_data/onnx_models/
rm *.onnx

from sparknlp.annotator import *
from sparknlp.base import *
#sparknlp doesn't like Volumes
tokenClassifier = BertForTokenClassification\
  .loadSavedModel("dbfs:"+onnxfile, spark)\
  .setInputCols(["document",'token'])\
  .setOutputCol("ner")\
  .setCaseSensitive(True)\
  .setMaxSentenceLength(128)

In [0]:
from synapse.ml.onnx import ONNXModel

onnx_ml = ONNXModel().setModelPayload(dummy_model_input)

print("Model inputs:" + str(onnx_ml.getModelInputs()))
print("Model outputs:" + str(onnx_ml.getModelOutputs()))